In [2]:
import matplotlib.pyplot as plt
import numpy as np

# O método da bissecção

## $ \S 1 $ Introdução

O método da bissecção é um procedimento geral para aproximação de zeros de uma
função  real $ f $. Seus únicos requerimentos são que a função seja contínua e
que tenhamos previamente identificado um intervalo $ [a, b] $ dentro do domínio
de $ f $ onde ela troca de sinal. Seus principais méritos são:
* _Confiabilidade_: ele sempre converge a um zero, satisfeitas as hipóteses
  acima.
* _Simplicidade_: ele é facilmente implementado, abrangente e não é necessário calcular a
  derivada da função.
* _Robustez_: pequenos erros de arredondamento (comparados à precisão desejada)
  não impactam seu resultado.

Por outro lado, o método não pode ser empregado para se localizar um zero de
multiplicidade par (como o da função $ x \mapsto x^2 $). Além disto, em muitos
casos é possível utilizar métodos cuja convergência é ainda mais rápida. Se
queremos obter um zero de uma única função, esta diferença de desempenho não é
relevante, porém a situação muda se o algoritmo precisa ser executado bilhões ou
trilhões de vezes.

## $ \S 2 $ Descrição do método da bissecção

Como acima, sejam $ f $ uma função real _contínua_ e $ [a, b] $ um intervalo
dentro do domínio de $ f $ tal que
$$
\boxed{\operatorname{sinal} f(a) \ne \operatorname{sinal} f(b)}
$$
No **método da bissecção** começamos tomando $ m $ como o **ponto médio** de $ a $ e $ b $,
$$
\boxed{m = \frac{a+b}{2}} 
$$
e avaliando $ f $ aí. Há três possibilidades:
* Se $ f(m)= 0 $, então $ m $ é um zero de $ f $ e podemos terminar.
* Se $ \operatorname{sinal} f(m)  \neq \operatorname{sinal} f(a) $, então $ f $
  troca de sinal em $ [a, m] $. Neste caso fazemos $ b \leftarrow m $.
* Se $ \operatorname{sinal} f(m) = \operatorname{sinal} f(a) $, então $ f $
  troca de sinal em $ [m, b] $. Neste caso fazemos $ a \leftarrow m $.

Nos dois últimos casos, podemos restringir nossa busca a um intervalo cujo
comprimento é a *metade* do anterior. O teorema do valor intermediário garante
que este subintervalo ainda contém um zero, portanto podemos aplicar o mesmo
procedimento a ele, usando os valores atualizados de $ a $ e $ b $.  Bissecções
sucessivas eventualmente nos levarão a um zero ou a um encaixotamento de um zero
por um subintervalo de comprimento tão pequeno quanto desejado.  Quando isto
acontecer, retornamos o ponto médio deste subintervalo como estimativa para o
zero.

📝 O método da bissecção também é conhecido como _busca binária_.

📝 Não é necessário _isolar_ um zero dentro do intervalo inicial, ou seja, o
método funciona mesmo que haja mais de um zero dentro de $ [a, b] $.

__Exemplo 1:__ Execute em seqüência as três células abaixo para ver uma animação
do método da bissecção aproximando um zero da função $ f(x) = x^3 - x - 3 $.
(Para facilitar a navegação posterior, depois você pode esconder as células
clicando sobre a barra correspondente à esquerda.) 

In [3]:
def print_solution(xs, ys, freq=1):
    """
    Given two arrays xs and ys of the same length, prints a table whose n-th
    line consists of three entries: the values of n, xs[n] and ys[n].
    Parameters:
        * The arrays xs and ys. 
        * A parameter freq used to print only one in every freq line. The
          first, second and last line are always printed. If freq == 0, then
          only these lines are printed.
    Returns: None.
    Prints: A header and the table described above.
    """
    def print_header():
        """
        Prints the table's header.
        """
        print("\n|       n      ", end="")
        print("    x_n            ", end="")
        print("    f(x_n)      |")
        print("|=================================================|")
        
    def print_line(n, x, y):
        """
        Pretty-prints n, x and y.
        """
        if n == 0:
            print(f"|       a", end="")
        elif n == 1:
            print(f"|       b", end="")
        else:
            print(f"|      {n - 1:02}", end="")
        print(f"    {x:15.8f}", end="")
        print(f"    {y:15.8f}   |")
    
    
    N = len(xs)
    if freq == 0:       # If freq == 0, print only first and last lines.
        freq = N - 1
    print_header()
    for n in range(0, 2):
        print_line(n, xs[n], ys[n])
    for n in range(2, N, freq):
        print_line(n, xs[n], ys[n])
    if n != N - 1:
        print_line(n, xs[N - 1], ys[N - 1])
    print("|_________________________________________________|\n")
        
    return None

In [33]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

def bisection_animation(f, a, b, N=4, title="", duration=0.75):
    """
    Displays an animation of the bisection method applied to a function.

    Parameters:
    * f: A continuous real function f.
    * a: The left endpoint of the interval [a, b] such that f(a)f(b) < 0.
    * b: The right endpoint of the interval [a, b] such that f(a)f(b) < 0.
    * N: The maximum number of iterations.
    * title: A title to be displayed at the top of the diagram.
    * duration: The duration of the pause between slides of the animation, in seconds.

    Returns:
    * xs: List of float, the estimates from the bisection method.
    * ys: List of float, the values of the function f at each of the estimates.
    * animation_html: IPython.display.HTML object containing the animation.
    """

    def iterate(a, b):
        """Applies a single step of the bisection method to the interval [a, b]."""
        m = 0.5 * (a + b)  # Midpoint of [a, b]
        if np.sign(f(a)) != np.sign(f(m)):  # [a, m] contains a zero
            return m, a, m
        else:  # [m, b] contains a zero
            return m, m, b

    cmap = plt.get_cmap("tab10")
    P = 200
    width = 1.75
    marker_size = 5
    domain = np.linspace(a, b, P)
    xs = [a, b]
    for _ in range(N):
        m, a, b = iterate(a, b)
        xs.append(m)
    ys = [f(x) for x in xs]

    # Lists containing the x and y coordinates for plotting vertical lines:
    xs_vert = [np.linspace(xs[n], xs[n], P) for n in range(N + 2)]
    ys_vert = [np.linspace(0, ys[n], P) for n in range(N + 2)]

    fig, ax = plt.subplots()
    ax.axhline(y=0.0, color='black', linestyle='-', lw=width)
    ax.plot(domain, f(domain), label="$ y = f(x) $", lw=width)
    ax.set_xlabel("$ x $")
    ax.set_ylabel("$ y $")
    ax.set_title(title)
    ax.grid(linestyle="dotted", alpha=0.5)
    ax.legend()

    lines = []

    for i in range(N + 2):
        color = cmap(i + 1)
        (line,) = ax.plot([], [], color=color, marker="x", mew=width)
        (vert_line,) = ax.plot([], [], linestyle='--', label=f"$ x_{i + 1} $", lw=width)
        (point,) = ax.plot([], [], color='black', marker="o", ms=marker_size)
        lines.append((line, vert_line, point))

    def init():
        for line, vert_line, point in lines:
            line.set_data([], [])
            vert_line.set_data([], [])
            point.set_data([], [])
        return [item for sublist in lines for item in sublist]

    def update(frame):
        if frame == 0:
            return [item for sublist in lines for item in sublist]
        n = frame - 1
        if n < N + 2:
            x = xs[n]
            y = ys[n]
            line, vert_line, point = lines[n]
            line.set_data([x], [0])
            vert_line.set_data(xs_vert[n], ys_vert[n])
            point.set_data([x], [y])
        return [item for sublist in lines for item in sublist]

    ani = FuncAnimation(fig, update, frames=range(N + 3), init_func=init,
                        blit=True, interval=duration * 1000)
    
    plt.close(fig)
    return xs, ys, HTML(ani.to_jshtml())

In [34]:
a = 0           # Extremidade esquerda do intervalo inicial, onde f vale -3.
b = 2           # Extremidade direita, onde f vale 3.
N = 5           # Número de iterações desejado.
pausa = 1.0     # Intervalo de tempo entre cada passo da animação, em segundos.
f = lambda x: x**3 - x - 3    # Função à qual o método será aplicado.
# Título a ser exibido no topo do diagrama:
titulo = "Método da bissecção para $ y = x^3 - x - 3, a = 0, b = 2 $."

xs, ys, ani = bisection_animation(f, a, b, N, titulo, pausa)
ani

<div align="center">
  <img src="fig_2-3_exemplo_1.png" alt="Exemplo do Método da Bissecção" width="600">
  <p><em>Figura: iterações do método da bissecção mostrando convergência para o zero</em></p>
</div>

In [5]:
print_solution(xs, ys)


|       n          x_n                f(x_n)      |
|=================================================|
|       a         0.00000000        -3.00000000   |
|       b         2.00000000         3.00000000   |
|      01         1.00000000        -3.00000000   |
|      02         1.50000000        -1.12500000   |
|      03         1.75000000         0.60937500   |
|      04         1.62500000        -0.33398438   |
|      05         1.68750000         0.11791992   |
|_________________________________________________|



## $ \S 3 $ Análise do desempenho do método da bissecção

Cada passo do método da bissecção corta o comprimento do intervalo anterior pela metade. Portanto dez passos reduzem o comprimento do intervalo original por um fator de $ 2^{10} > 1\,000 $. Mais geralmente, temos o seguinte resultado.

**Teorema 3.1 (análise do método da bissecção):** _Pelo método da bissecção, o
número de iterações necessário para se localizar um zero dentro do intervalo
inicial $ [a, b] $ com erro menor que ou igual a $ \varepsilon > 0 $ é dado
por:_
\begin{equation*}
    \boxed{\left\lceil{\lg\bigg(\frac{b-a}{\varepsilon}\bigg)}\right\rceil
    \quad \text{onde $ \lg = \log_2 $}}
\end{equation*}

**Prova:** Começando com o intervalo $ [a,b] $ original (primeiro passo),
sabemos que a cada iteração há um zero dentro do intervalo atual. Após o $ n
$-ésimo passo, o comprimento de $ [a, b] $ terá sido reduzido pelo fator de $
2^{n-1} $. Mas como o resultado do método é o ponto médio do último intervalo
considerado, o que importa é a _metade_ do seu comprimento, já que esta é uma
cota superior para a distância do ponto médio a um zero. Logo $ n $ deve ser
grande o suficiente de modo que
$$
    \frac{b-a}{2^n} \le \varepsilon\,, \quad \text{ou equivalentemente,} \quad 
    \lg \left( \frac{b-a}{\varepsilon} \right) \le n\,.
$$
O menor inteiro que satisfaz esta desigualdade é o teto desta última expressão, como
no enunciado. $\ \ \blacksquare $

📝 Informalmente, o resultado acima implica que para cada dígito adicional de
precisão, precisamos de $ \lg 10 \approx 3.32 $ iterações a mais. Este
desempenho deve ser comparado com o da busca incremental, que depende
_linearmente_ de $ \frac{1}{\varepsilon} $; portanto para melhorar a precisão
desta por um dígito decimal, precisamos _multiplicar_ o número de iterações por
$ 10 $.

## $ \S 4 $ Critérios de parada e observações

Os critérios de parada que podemos utilizar são essencialmente os mesmos em
todos os métodos para determinação de zeros:

1. A distância da estimativa atual a um zero é menor que um $ \varepsilon > 0 $ pré-escolhido.
2. O módulo do valor da função na estimativa atual é menor que um $ \delta > 0 $ escolhido previamente.
3. O número de iterações excede uma cota prefixada.

Além destas, podemos também considerar versões _relativas_ (em vez de absolutas) de 1 e 2:

1'. A distância da estimativa atual ao zero relativamente ao comprimento do
   intervalo original é menor que um $ \varepsilon $ pré-especificado:
   $$
     \frac{b_n - a_n}{2(b - a)} = \frac{1}{2^n} < \varepsilon \,.
   $$
   Portanto no caso do método da bissecção, este critério é essencialmente
   equivalente ao terceiro acima.

2'. O módulo do valor da função na estimativa $ m $ atual é menor que
   $ \delta $ relativamente aos valores originais $ f(a) $ e $ f(b) $, i.e.:
   $$
     \frac{|f(m)|}{\min\left\{|f(a)|\,,\,|f(b)|\right\}} < \delta.
   $$

Na implementação abaixo, escolhemos fazer o procedimento terminar assim que
algum dos critérios 1 ou 3 for satisfeito.

⚠️ Para que possamos sequer aplicar o método da bissecção a uma função, antes é
necessário ter obtido um intervalo onde ela troca de sinal. Este requerimento
preliminar geralmente pode ser satisfeito com a inspeção do gráfico da função ou
com uma busca incremental (conforme explicado no caderno anterior).

⚠️ Por definição, $ a $ é um **pólo** de ordem $ k $ de uma função $ f $ se ele
é um zero de multiplicidade $ k $ de $ 1 / f(x) $. Se $ f $ tiver um pólo de
ordem ímpar num ponto, como o da função $ x \mapsto \frac{1}{x} $ em $ x = 0 $,
então o método da bissecção poderá confundi-lo com um zero. Execute a animação
abaixo para entender o que acontece nestes casos.

In [24]:
a = -0.15       # Extremidade esquerda do intervalo inicial.
b = 0.22        # Extremidade direita.
N = 6           # Número de iterações desejado.
pausa = 0.75    # Intervalo de tempo entre cada passo da animação, em segundos.
f = lambda x: 1 / x   # Função à qual o método será aplicado.
# Título a ser exibido no topo do diagrama:
titulo = "Método da bissecção para $ y = 1/x, a = -0.15, b = 0.22 $;\n"\
         "confundindo um pólo de ordem ímpar com um zero."

xs, ys, ani = bisection_animation(f, a, b, N, titulo, pausa)
ani

In [9]:
print_solution(xs, ys)


|       n          x_n                f(x_n)      |
|=================================================|
|       a        -0.15000000        -6.66666667   |
|       b         0.22000000         4.54545455   |
|      01         0.03500000        28.57142857   |
|      02        -0.05750000       -17.39130435   |
|      03        -0.01125000       -88.88888889   |
|      04         0.01187500        84.21052632   |
|      05         0.00031250      3200.00000000   |
|      06        -0.00546875      -182.85714286   |
|_________________________________________________|



## $ \S 5 $ Implementação do método da bissecção

In [7]:
def bisection(f, a, b, eps=1e-3, max_iter=100):
    """
    Approximate a zero of a function using the bisection method.
    Parameters:
        * f (function): A real continuous function.
        * a, b (floats): Interval such that f(a) * f(b) < 0.
        * eps (float): Error tolerance.
        * max_iter (int): Maximum iterations.
    Returns:
        * List of estimates and corresponding function values.
    """
    from numpy import sign

    xs, ys = [a, b], [f(a), f(b)]
    iterations = 0

    while (b - a) >= eps and iterations < max_iter:
        m = (a + b) / 2
        xs.append(m)
        ys.append(f(m))
        
        if f(m) == 0:
            break
        elif sign(f(a)) != sign(f(m)):
            b = m
        else:
            a = m

        iterations += 1

    print(f"Approximate zero at {m:.9f} after {iterations} iterations with error {b - a:.9f}.")
    print(f"f({m:.9f}) = {f(m):.9f}")

    return xs, ys

__Problema 1:__ Obtenha uma estimativa para $ \pi $, correta até a sexta casa
decimal, utilizando o método da bissecção para aproximar o primeiro zero
positivo da função seno. 

*Solução:*

__Problema 2:__ Seja
$$ f(x) = (x + 2)(x + 1)^2(x - 1)(x - 2)^3\,.
$$
A qual zero de
$ f $ o método da bissecção converge quando aplicado a esta função, com
intervalo inicial $ [-3, 3] $?


_Solução:_

__Problema 3:__ 

(a) Mostre que a equação $ \frac{2}{x} + \frac{5}{x^3} = x $ admite exatamente
duas raízes reais, uma positiva, outra negativa.

(b) Determine estas raízes com uma precisão melhor que $ 10^{-6} $ usando o
método da bissecção.

(c) Aplicando-se o método da bissecção com intervalo inicial $ [-2, 1] $ à função
$$
    f(x) = \frac{2}{x} + \frac{5}{x^3} - x
$$
é produzida uma seqüência que parece convergir a $ 0 $, mas esta função sequer
está definida aí. Justifique.

*Solução:*

## $ \S 6 $ Problemas

__Problema 4:__ Utilizando o computador apenas como uma calculadora:

(a) Aplique quatro iterações do método da bissecção para estimar $ \sqrt[3]{2} $.
(Na primeira iteração, você deve fornecer como estimativa o ponto médio
do intervalo original.)
 *Dica:* Considere a função $ f(x) = x^3 - 2 $.

(b) Use o Teorema 3.1 para calcular o número mínimo de iterações necessárias para garantir uma precisão de $ 10^{-5} $.

*Solução:*

__Problema 5:__ Usando o computador:

(a) Encontre a menor raiz real positiva da equação
$$
x^3 - 3.45x^2 - 5.72 x + 6.31 = 0
$$
com uma precisão melhor que $ 10^{-3} $ usando o método da bissecção.

(b) Considerando o intervalo original que você utilizou, quantos passos são necessários?

*Solução:*

__Problema 6:__

(a) Mostre que a equação $ xe^x = 1 $ possui uma única raiz em $ \mathbb{R} $.
*Dica:* O que se pode dizer sobre o sinal da derivada de $ f(x) = xe^x -
1 $?

(b) Mostre que esta raiz se encontra no intervalo $ [0.5, 1] $.

(c) Estime esta raiz com erro máximo de $ 10^{-5} $, usando o método da bissecção.

(d) Quantos passos são necessários para garantir esta precisão?

*Solução:*

__Problema 7:__ Seja $ y = \tan x $ a função tangente. Recorde que 
$$
\tan\left(\frac{\pi}{4}\right) = 1 \quad \text{e} \quad
\tan\left(\frac{2\pi}{3}\right) = -\sqrt{3}
$$
Usando o computador, aplique o método da bissecção a $ f = \tan $ no
intervalo $ \left[\frac{\pi}{4}\,,\,\frac{2\pi}{3}\right] $ e explique o
resultado.

*Solução:*

__Problema 8:__ Resolva o Problema 5 para a equação $ x^2 + \ln x = 0 $ ($ x > 0 $).

*Solução:*

**Problema 9:** Encontre uma raiz de cada uma das equações abaixo com erro $ < 10^{-5} $. *Dica:* Antes de aplicar o método da bissecção, é necessário encaixotar um zero. Para isto, utilize a análise gráfica ou a busca incremental. Os procedimentos correspondentes estão na [última seção](#auxiliar) deste caderno.

(a) $ x \ln x = 1 $ ($ x > 0 $).

(b) $ \cos x = x^2 $.

(c) $ x^5 - 3x^4 - 6x^3 + 4x^2 + 5x - 3 = 0 $.

(d) $ \tan x = x + 2 e^x $ $\big( x \not\in \frac{\pi}{2} + \pi \mathbb Z \big)$.

(e) $ \cos x \cosh x = 2 $, onde por definição $ \cosh x = \frac{e^x + e^{-x}}{2} $.

*Solução:*

## $ \S 7 $ ⚡ Análise da convergência do método da bissecção

**Teorema 7.1:** _Seja $ f \colon [a, b] \to \mathbb R $ uma função_
contínua _tal que $ f(a) $ e $ f(b) $ têm sinais opostos. Sejam $ [a_n, b_n] $ o
intervalo resultante após a $ n $-ésima iteração e $ x_n $ seu ponto médio, com
$ [a_1, b_1] = [a, b] $.  Então ou o método da bissecção produz um zero exato
após um número finito de passos, ou as seqüências $ (a_n) $, $ (x_n) $ e  $
(b_n) $ construídas por ele convergem a um mesmo zero de_ $ f $.

**Prova:** Suponha que o método da bissecção não termine após um número finito de passos. Observe que
$$
a = a_1 \le a_2 \le a_3 \le \cdots a_n \le \cdots \le \cdots \le b_n \le \cdots b_3 \le b_2 \le b_1 = b
$$
e $ (b_n - a_n) \to 0 $. Mais detalhadamente, por construção valem:
1. A seqüência $ (a_n) $ é crescente e limitada superiormente por $ b $.
2. A seqüência $ (b_n) $ é decrescente e limitada inferiormente por $ a $
3. $ a_n < b_n $ e $ b_n - a_n = \frac{(b - a)}{2^{n - 1}} $ para todo $ n $.

Pela
[completude](https://en.wikipedia.org/wiki/Completeness_of_the_real_numbers) dos
números reais, qualquer seqüência crescente e limitada superiormente converge (o
limite é a _menor cota superior_ do conjunto de valores desta seqüência).
Analogamente, qualquer seqüência decrescente e limitada inferiormente converge
(à _maior cota inferior_ do conjunto de valores dela).

Seja $ \zeta $ o limite de $ (a_n) $. Então, como 
$$
b_n - a_n = \frac{b - a}{2^{n - 1}}
$$
e como $ (b_n) $ converge, deduzimos que o limite desta também é $ \zeta $.
Fazendo $ n \to \infty $ na desigualdade
$$
f(a_n) f(b_n) < 0
$$
e usando a continuidade de $ f $, concluímos que
$$
\lim_n \big[f(a_n) f(b_n)\big] = \big[\lim_n f(a_n)\big] \big[\lim_n f(b_n)\big]
= f(\lim_n a_n) f(\lim_n b_n) = f(\zeta) f(\zeta) = f(\zeta) ^2 \le 0\,.
$$
Isto só é possível se $ f(\zeta) = 0 $, ou seja, se $ \zeta $ é um zero de $ f
$. Finalmente, $ (x_n) $ também converge a $ \zeta $, já que por definição vale
$$
a_n < x_n < b_n \quad \text{para todo $ n $}\,. \tag*{$ \blacksquare $}
$$

📝 A completude de $ \mathbb R $ é uma propriedade fundamental do conjunto dos
números reais, normalmente tomada como um axioma no desenvolvimento da teoria
destes.  Note que o conjunto $ \mathbb Q $ dos números  racionais não tem esta
propriedade. Por exemplo, considere a seqüência 
$$
3.1, \quad 3.14, \quad 3.141, \quad 3.1415, \quad \cdots
$$
cujo $ n $-ésimo termo é o truncamento de $ \pi $ até o $ n $-ésimo dígito
decimal. Esta seqüência é crescente e limitada, mas ela não converge em $
\mathbb Q $. Similarmente, o conjunto dos números irracionais tampouco é
completo.

## $ \S 8 $ Procedimentos auxiliares<a name="auxiliar"></a>

In [12]:
def incremental_search(f, a, b, h):
    """
    Incremental search for a sign change in f within [a, b] with step size h.
    
    Parameters:
    * f (function): Real continuous function.
    * a (float): Start of interval (a < b).
    * b (float): End of interval.
    * h (float): Step size (h > 0).
    
    Returns:
    * (c, d, iterations) (float, float, int): 
      The number of iterations performed and values c and d for x such that:
      * a <= c < d <= b
      * d - c <= h
      * sign f(c) != sign f(d)
    """
    from numpy import sign

    # Checking for unexpected arguments:
    if sign(h) != 1:
        raise ValueError("The step size should be positive!")
    if a >= b:
        raise ValueError("a should be less than b!")

    c = a
    d = a + h
    iterations = 1

    # Search for a change of signs in steps of size h:
    while sign(f(c)) == sign(f(d)) and c != b:
        c = d
        d = min(d + h, b)
        iterations += 1

    if c == b:  # Search inconclusive
        return None, None, iterations
    else:       # Search successful
        return round(c, 10), round(d, 10), iterations

In [13]:
def plot_functions(a, b, N, *fs):
    """
    Parameters:
        * The left and right endpoints a and b (float) of the plot interval.
        * The number N (int) of points used in each plot.
        * Any quantity of functions defined on [a, b].
    Output: None.
    Displays:
        * The graphs of the given functions in a single diagram.
    """
    import matplotlib.pyplot as plt
    import numpy as np
    

    line_width = 1.5  # Set the line width a little thicker
    # Create sample of N + 1 values of x in [a, b]:
    xs = np.linspace(a, b, N + 1)
    # Plot the graph of each function:
    for n, f in enumerate(fs):
        plt.plot(xs, f(xs), lw=line_width, label=f"$ f_{n} $")

    # Draw thicker x- and y-axes:
    plt.axhline(y=0.0, color='black', linestyle='-', lw=line_width)
    plt.axvline(x=0.0, color='black', linestyle='-', lw=line_width)

    # Improve the design:
    plt.xlabel("$ x $")  # Label for the x-axis
    plt.ylabel("$ y $")  # Label for the y-axis
    plt.grid(True)    # Draw a rectangular grid.
    plt.legend()  # Legend to help distinguish the graphs

    return None